In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms,models
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
device="cuda" if torch.cuda.is_available() else "CPU"

In [ ]:
TRAIN_DIR="../input/dogs-and-cats-datasets/train/train"
TEST_DIR="../input/dogs-and-cats-datasets/test/test"
file_list_train=os.listdir(TRAIN_DIR)
_file_list_test=os.listdir(TEST_DIR)

In [ ]:
_file_list_test

In [ ]:
!pip install natsort

In [ ]:
from natsort import natsorted
file_list_test=[]
for path in natsorted(_file_list_test):
    file_list_test.append(path)

In [ ]:
file_list_test

In [ ]:
ROWS=64
COLS=64
CHANNELS=3

In [ ]:
cat_files=[file_name for file_name in file_list_train if "cat" in file_name]
dog_files=[file_name for file_name in file_list_train if "dog" in file_name]

In [ ]:
print(len(cat_files))
print(len(dog_files))

In [ ]:
from sklearn.model_selection import train_test_split
train_cat_files, val_cat_files = train_test_split(cat_files, test_size=0.2)
train_dog_files, val_dog_files = train_test_split(dog_files, test_size=0.2)

In [ ]:
train_transform=transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.5, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
])

val_transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
])

test_transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
])

In [ ]:
class CatDogDataset(Dataset):
    def __init__(self,file_list,dir,transform=None):
        self.file_list=file_list
        self.dir=dir
        self.transform=transform
        if "dog" in self.file_list[0]:
            self.label=1
        else:
            self.label=0
    def __len__(self):
        return len(self.file_list)
    def __getitem__(self,idx):
        file_path =os.path.join(self.dir,self.file_list[idx])
        img=Image.open(file_path)
        if self.transform is not None:
            img =self.transform(img)
        return img,self.label

In [ ]:
dir_path="../input/dogs-and-cats-datasets/train/train"
train_cat_dataset=CatDogDataset(train_cat_files,dir_path,transform=train_transform)
train_dog_dataset=CatDogDataset(train_dog_files,dir_path,transform=train_transform)
val_cat_dataset=CatDogDataset(val_cat_files,dir_path,transform=val_transform)
val_dog_dataset=CatDogDataset(val_dog_files,dir_path,transform=val_transform)

In [ ]:
print(len(train_cat_dataset))
print(len(train_dog_dataset))
print(len(val_cat_dataset))
print(len(val_dog_dataset))

In [ ]:
train_cat_dataset.file_list=train_cat_dataset.file_list[:100]
train_dog_dataset.file_list=train_dog_dataset.file_list[:100]
val_cat_dataset.file_list=val_cat_dataset.file_list[:10]
val_dog_dataset.file_list=val_dog_dataset.file_list[:10]

In [ ]:
train_cat_dog_dataset=ConcatDataset([train_cat_dataset,train_dog_dataset])
val_cat_dog_dataset=ConcatDataset([val_cat_dataset,val_dog_dataset])

In [ ]:
n_samples = len(train_cat_dog_dataset) 
train_size = int(n_samples * 0.8)
subset1_indices = list(range(0,train_size))
subset2_indices = list(range(train_size,n_samples))
from torch.utils.data.dataset import Subset
train_dataset = Subset(train_cat_dog_dataset, subset1_indices)
val_dataset   = Subset(train_cat_dog_dataset, subset2_indices)

In [ ]:
train_dataloader=DataLoader(train_cat_dog_dataset,batch_size=32,shuffle=True)
validation_dataloader=DataLoader(val_cat_dog_dataset,batch_size=32,shuffle=True)

In [ ]:
data_iter=iter(train_dataloader)
imgs,labels=data_iter.next()
img=imgs[0]
img_prmute=img.permute(1,2,0)
img_permute=0.5*img_prmute+0.5
img_permute=np.clip(img_permute,0,1)
plt.imshow(img_permute)

In [ ]:
labels.size()

In [ ]:
imgs.size()

In [ ]:
class CNN(nn.Module):
    def __init__(self,num_classes):
        super().__init__()
        self.features=nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=5,padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=256,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
        )
        self.classifier=nn.Linear(in_features=32*32*128,out_features=num_classes)
    def forward(self,x):
        x=self.features(x)
        x=x.view(x.size(0),-1)
        x=self.classifier(x)
        #x=nn.ReLU(x)
        return x

In [ ]:
model=models.vgg16(pretrained=True)
#model=models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad=False
model.classifier[6] = nn.Linear(in_features=4096, out_features=2)

In [ ]:
params_to_update = []

update_params_name = ['classifier.6.weight', 'classifier.6.bias']

for name, param in model.named_parameters():
    if name in update_params_name:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

In [ ]:
#model=CNN(2)

In [ ]:
model.to(device)

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(params=params_to_update,lr=0.001,momentum=0.9)
#optimizer=optim.Adam(model.parameters(),lr=0.001,weight_decay=5e-4)

In [ ]:
num_epochs=5
losses=[]
accs=[]
val_losses=[]
val_accs=[]
for epoch in range(num_epochs):
    running_loss=0.0
    running_acc=0.0
    for imgs,labels in train_dataloader:
        imgs=imgs.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(imgs)
        loss=criterion(output,labels)
        loss.backward()
        running_loss+=loss.item()
        pred=torch.argmax(output,dim=1)
        #print(pred)
        running_acc+=torch.mean(pred.eq(labels).float())
        optimizer.step()
    running_loss /=len(train_dataloader)
    running_acc /=len(train_dataloader)
    losses.append(running_loss)
    accs.append(running_acc)

    val_running_loss=0.0
    val_running_acc=0.0
    for val_imgs,val_labels in validation_dataloader:
        val_imgs=val_imgs.to(device)
        val_labels=val_labels.to(device)
        val_output=model(val_imgs)
        val_loss=criterion(val_output,val_labels)
        val_running_loss+= val_loss.item()
        val_pred=torch.argmax(val_output,dim=1)
        val_running_acc+=torch.mean(val_pred.eq(val_labels).float())
    val_running_loss/=len(validation_dataloader)
    val_running_acc/=len(validation_dataloader)
    val_losses.append(val_running_loss)
    val_accs.append(val_running_acc)
    print("epoch:{},        loss:{},        acc:{},     val loss:{},        val acc:{}".format(epoch,running_loss,running_acc,val_running_loss,val_running_acc))

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.plot(losses,label="train loss")
plt.plot(val_losses,label="validation loss")
plt.legend()

In [ ]:
plt.style.use("ggplot")
plt.plot(accs,label="train acc")
plt.plot(val_accs,label="validation acc")
plt.legend()

* この下からtestの検証

In [ ]:
test_files=[file_name for file_name in file_list_test]

In [ ]:
dir_path="../input/dogs-and-cats-datasets/test/test"
test_dataset=CatDogDataset(test_files,dir_path,transform=test_transform)

In [ ]:
test_dataloader=DataLoader(test_dataset,shuffle=False)

In [ ]:
"""
data_iter=list(iter(test_dataloader))
imgs,labels=data_iter[0]
img=imgs[0]
img_prmute=img.permute(1,2,0)
img_permute=0.5*img_prmute+0.5
img_permute=np.clip(img_permute,0,1)
plt.imshow(img_permute)
"""

In [ ]:
result=[]
for imgs,labels in test_dataloader:
    imgs=imgs.to(device)
    output=model(imgs)
    #pred=torch.argmax(output,dim=1)
    pred= F.softmax(output, dim=1)[:, 1]
    pred2 = pred.to('cpu').detach().numpy().copy()
    result.append(pred2)

In [ ]:
print(output)
print(pred)
print(pred2)
print(result)

In [ ]:
result2=np.array(result)

In [ ]:
result2

In [ ]:
result3 = result2.flatten()

In [ ]:
result3

In [ ]:
result4=result3.tolist()

In [ ]:
result4

In [ ]:
import pandas as pd
submission=pd.DataFrame({"label":result4})
submission.index = submission.index + 1
submission.to_csv(("./submission_4.csv"),index_label=["id"])

In [ ]:
len(submission)